# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [89]:
from azureml.core import Workspace, Experiment, Dataset, Model
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig
import numpy as np
import joblib, pickle

In [1]:
from azureml.core import Workspace, Experiment, Dataset, Model
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig
import numpy as np
import joblib, pickle

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
subscription_id = '3ec87f2b-33cf-4aed-b36f-750175813524'
resource_group = 'aml-quickstarts-128089'
workspace_name = 'quick-starts-ws-128089'

ws = Workspace.get(name="quick-starts-ws-128089")
exp = Experiment(workspace=ws, name="Ranga")

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='mpg')
ds = dataset.to_pandas_dataframe()

ds = ds.replace('?',0)
ds.dropna(inplace=True)

ds.drop(columns=['car name'],inplace=True)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code F6L8BGLY2 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [3]:
# TODO: Put your automl config here
automl_config = AutoMLConfig(
    experiment_timeout_minutes=20,
    task='regression',
    primary_metric='r2_score',
    training_data=ds,
    label_column_name='mpg',
    n_cross_validations=3)

In [4]:
# TODO: Submit your experiment
autoexp = Experiment(workspace=ws,name='AutoRanga')
remote_run = autoexp.submit(automl_config)
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

WARNING - Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'




****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       DONE
DESCRIPTION:  High cardinality features were detected in your inputs and handled.
              Learn more about high cardinality feature handling: https://aka.ms/AutomatedMLFeaturization
DETAILS:      High cardinality features refer to columns that contain a large percentage of unique values.
+---------------------------------+---------------------------------+
|Column name                      |Column Content Type              |
+=============================

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


****************************************************************************************************

****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
****************************************************************************************************

 ITERATION   PIPELINE                                       DURATION      METRIC      BEST
         0   MaxAbsScaler LightGBM                          0:00:34       0.7822    0.7822
         1   MaxAbsScaler XGBoostRegressor                  0:00:26       0.8192    0.8192
         2   StandardScalerWrapper XGBoostRegressor         0:00:26       0.8308    0.8308
         3   StandardScalerWrapper RandomForest             0:00:26      

{'runId': 'AutoML_4798bf11-a0a6-4219-af08-bb9b29393c00',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2020-11-27T14:25:38.628329Z',
 'endTimeUtc': '2020-11-27T14:47:08.702042Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'r2_score',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"AutoRanga","subscription_id":"3ec87f2b-33cf-4aed-b36f-750175813524","resource_group":"aml-quickstarts-128089","workspace_name":"quick-starts-ws-128089","region":"southcentralus","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","_local_managed_run_id":null,"many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"r2_score","task_type":"regression","data_script":null,"validation_size":0.0,"n_cross_validations":3,"y_min":null,"y_max":null

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [5]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [6]:
best,fitted = remote_run.get_output()
print(best)

Run(Experiment: AutoRanga,
Id: AutoML_4798bf11-a0a6-4219-af08-bb9b29393c00_43,
Type: None,
Status: Completed)


In [7]:
print(fitted)

RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=None,
                                                             enable_feature_sweeping=None,
                                                             feature_sweeping_config=None,
                                                             feature_sweeping_timeout=None,
                                                             featurization_config=None,
                                                             force_text_dnn=None,
                                                             is_cross_validation=None,
                                                             is_onnx_compatible=None,
                                                             logger=None,
                                                             observer=None,
                                         

In [8]:
#TODO: Save the best model
filename = 'best_automl_model.sav'
# pickle.dump(fitted, open(filename, 'wb'))

joblib.dump(fitted,filename)

['best_automl_model.sav']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [60]:
registeredfinal = Model.register(workspace=ws,model_path='./best_automl_model.sav',model_name='registered.sav')

Registering model registered.sav


In [61]:
path = Model.get_model_path('registered.sav')
joblib.load(path)

RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=None,
                                                             enable_feature_sweeping=None,
                                                             feature_sweeping_config=None,
                                                             feature_sweeping_timeout=None,
                                                             featurization_config=None,
                                                             force_text_dnn=None,
                                                             is_cross_validation=None,
                                                             is_onnx_compatible=None,
                                                             logger=None,
                                                             observer=None,
                                         

In [181]:
#Create inference config
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

env = Environment.get(workspace=ws,name='AzureML-AutoML')

inferencefinal = InferenceConfig(entry_script='score_11_28.py',environment=env) 

#Deploy model and check service state
from azureml.core.webservice import LocalWebservice, AciWebservice


deployfinal = AciWebservice.deploy_configuration()
svcs = Model.deploy(workspace=ws,name='feeh',models=[registeredfinal],inference_config=inferencefinal,deployment_config=deployfinal)
svcs.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running....................................................................

KeyboardInterrupt: 

In [ ]:
print(svcs.state)

In [ ]:
print(svcs.scoring_uri)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [331]:
import requests
import json

# URL for the web service
scoring_uri = 'http://caa9be12-aa84-41f1-877f-36ffadc75841.southcentralus.azurecontainer.io/score'


data = {'data':[{
    'cylinders':4,
    'displacement':85,
    'horsepower':65,
    'weight':2110,
    'acceleration':19,
    'model yr':80,
    'origin':3}
]
        }

input_data = json.dumps(data)

headers = {'Content-Type': 'application/json'}


# # Make the request and display the response
result = requests.post(scoring_uri,input_data,headers=headers)
print("prediction:",result.text)

In [334]:
model_path = Model.get_model_path('registered.sav')
mod = joblib.load(model_path)

TODO: In the cell below, print the logs of the web service and delete the service

In [150]:
print(svcs.get_logs())

2020-11-27T17:08:49.0235018Z stdout F 2020-11-27T17:08:48,651390700+00:00 - iot-server/run 
2020-11-27T17:08:49.1018981Z stdout F 2020-11-27T17:08:48,682097700+00:00 - gunicorn/run 
2020-11-27T17:08:49.1343385Z stdout F 2020-11-27T17:08:48,781035800+00:00 - rsyslog/run 
2020-11-27T17:08:49.3385051Z stdout F 2020-11-27T17:08:48,979933000+00:00 - nginx/run 
2020-11-27T17:08:49.4211127Z stderr F /usr/sbin/nginx: /azureml-envs/azureml_7cf2b5eebfc9a832d561ce064cfbc38b/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2020-11-27T17:08:49.4211127Z stderr F /usr/sbin/nginx: /azureml-envs/azureml_7cf2b5eebfc9a832d561ce064cfbc38b/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2020-11-27T17:08:49.4251177Z stderr F /usr/sbin/nginx: /azureml-envs/azureml_7cf2b5eebfc9a832d561ce064cfbc38b/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2020-11-27T17:08:49.4251177Z stderr F /usr/sbin/nginx:

In [ ]:
svcs.delete()